In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from datetime import datetime
from matplotlib import font_manager, rcParams

# 프로젝트 경로 설정
PROJECT_ROOT = Path.cwd().parent
PLOTS_DIR = PROJECT_ROOT / "artifacts" / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# 한글 폰트 설정
FONT_PATH = PROJECT_ROOT / "assets" / "fonts" / "NanumGothic-Regular.ttf"
if FONT_PATH.exists():
    font_manager.fontManager.addfont(str(FONT_PATH))
    rcParams['font.family'] = 'NanumGothic'
    rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
    print(f"✓ 한글 폰트 설정 완료: {FONT_PATH.name}")
else:
    print(f"⚠ 폰트 파일을 찾을 수 없습니다: {FONT_PATH}")


✓ 한글 폰트 설정 완료: NanumGothic-Regular.ttf


In [2]:
def save_plot(fig, name: str, prefix: str = ""):
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    safe_name = name.replace(" ", "_")
    filename = f"{prefix}{ts}_{safe_name}.png" if prefix else f"{ts}_{safe_name}.png"
    fig.savefig(PLOTS_DIR / filename, dpi=300, bbox_inches="tight")
    plt.close(fig)

In [3]:
train_df = pd.read_csv('/root/ml_project/first-ml-project-refactor/data/train.csv')
test_df = pd.read_csv('/root/ml_project/first-ml-project-refactor/data/test.csv')
subway_df = pd.read_csv('/root/ml_project/first-ml-project-refactor/data/subway_feature.csv')
bus_df = pd.read_csv('/root/ml_project/first-ml-project-refactor/data/bus_feature.csv')


In [22]:
print(train_df.shape)
print(test_df.shape)

(1118822, 52)
(9272, 51)


In [4]:
print(train_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [5]:
train_df.head()

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,130000
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,117000


In [25]:
test_df.isnull().sum().sort_values(ascending=False)

k-135㎡초과                  9270
해제사유발생일                   9060
단지소개기존clob                8718
k-등록일자                    8554
k-홈페이지                    7876
고용보험관리번호                  7453
세대전기계약방법                  6642
k-팩스번호                    6606
k-단지분류(아파트,주상복합등등)        6582
k-시행사                     6580
k-건설사(시공사)                6579
k-전체동수                    6577
k-전화번호                    6576
청소비관리형태                   6573
경비비관리형태                   6573
단지승인일                     6568
건축면적                      6565
k-복도유형                    6564
k-사용검사일-사용승인일             6563
주차대수                      6563
k-전용면적별세대현황(60㎡~85㎡이하)    6562
k-85㎡~135㎡이하              6562
k-관리비부과면적                 6562
k-연면적                     6562
k-주거전용면적                  6562
k-전용면적별세대현황(60㎡이하)        6562
k-전체세대수                   6562
k-관리방식                    6562
k-수정일자                    6562
기타/의무/임대/임의=1/2/3/4       6562
사용허가여부                    6562
관리비 업로드                   6562
좌표X     

In [6]:
# 결측치 내림차순 정렬

train_df.isnull().sum().sort_values(ascending=False)

k-135㎡초과                  1118495
해제사유발생일                   1112839
k-등록일자                    1107832
단지소개기존clob                1050240
k-홈페이지                    1005647
고용보험관리번호                   913304
세대전기계약방법                   878747
k-팩스번호                     872742
k-시행사                      871254
청소비관리형태                    871178
k-건설사(시공사)                 871058
경비비관리형태                    870988
k-단지분류(아파트,주상복합등등)         870691
k-전체동수                     870630
단지승인일                      870286
k-전화번호                     870274
k-복도유형                     869890
건축면적                       869714
주차대수                       869714
k-사용검사일-사용승인일              869696
좌표X                        869670
좌표Y                        869670
단지신청일                      869625
k-주거전용면적                   869608
k-전용면적별세대현황(60㎡~85㎡이하)     869608
k-전용면적별세대현황(60㎡이하)         869608
k-85㎡~135㎡이하               869608
k-수정일자                     869608
k-연면적                      869563
k-관리비부과면적     

In [ ]:
# 결측치 비율 시각화 코드
datasets = [
    ("train", train_df),
    ("test", test_df), 
]

for name, df in datasets:
    missing_count = df.isnull().sum()
    missing_pct = (missing_count / len(df) * 100).round(2)
    missing_summary = (
        pd.DataFrame({"missing_count": missing_count, "missing_pct": missing_pct})
        .sort_values("missing_count", ascending=True)
    )

    fig, ax = plt.subplots(figsize=(12, max(6, len(missing_summary) * 0.3)))
    ax.barh(missing_summary.index, missing_summary["missing_count"], color="#1f77b4")
    ax.set_title(f"{name.capitalize()} Missing Values by Column")
    ax.set_xlabel("Missing Count")
    ax.set_ylabel("Columns")

    for i, (cnt, pct) in enumerate(zip(missing_summary["missing_count"], missing_summary["missing_pct"])):
        ax.text(cnt, i, f"{pct}%", va="center", ha="left", fontsize=9, color="#333")

    fig.tight_layout()
    save_plot(fig, f"{name}_missing_values_bar", prefix="eda_")
    plt.show()

In [ ]:
# 결측치가 없는 컬럼들 추출 및 분포 확인
no_missing_cols = train_df.columns[train_df.isnull().sum() == 0].tolist()
print(f"결측치가 없는 컬럼 개수: {len(no_missing_cols)}")
print(f"컬럼 목록: {no_missing_cols}\n")

# 수치형 컬럼만 추출
numeric_no_missing = train_df[no_missing_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"그 중 수치형 컬럼: {numeric_no_missing}\n")

# 수치형 컬럼 분포 시각화
if numeric_no_missing:
    n_cols = len(numeric_no_missing)
    n_rows = (n_cols + 2) // 3  # 3열 그리드
    
    fig, axes = plt.subplots(n_rows, 3, figsize=(15, n_rows * 4))
    axes = axes.flatten() if n_cols > 1 else [axes]
    
    for idx, col in enumerate(numeric_no_missing):
        ax = axes[idx]
        train_df[col].hist(bins=50, ax=ax, color='steelblue', edgecolor='black', alpha=0.7)
        ax.set_title(f'{col} 분포')
        ax.set_xlabel(col)
        ax.set_ylabel('빈도')
        ax.grid(axis='y', alpha=0.3)
        
        # 기본 통계량 표시
        mean_val = train_df[col].mean()
        median_val = train_df[col].median()
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=1.5, label=f'평균: {mean_val:.1f}')
        ax.axvline(median_val, color='green', linestyle='--', linewidth=1.5, label=f'중앙값: {median_val:.1f}')
        ax.legend(fontsize=8)
    
    # 빈 subplot 제거
    for idx in range(n_cols, len(axes)):
        fig.delaxes(axes[idx])
    
    fig.tight_layout()
    save_plot(fig, "no_missing_columns_distribution", prefix="eda_")
    plt.show()

# 기본 통계량 출력
print("\n=== 기본 통계량 ===")
print(train_df[numeric_no_missing].describe())

결측치가 없는 컬럼 개수: 12
컬럼 목록: ['시군구', '본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명', '등기신청일자', '거래유형', '중개사소재지']

그 중 수치형 컬럼: ['본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도']


=== 기본 통계량 ===
                본번           부번      전용면적(㎡)           계약년월          계약일  \
count  9272.000000  9272.000000  9272.000000    9272.000000  9272.000000   
mean    615.101920     5.676553    75.414506  202307.786238    14.967429   
std     592.812553    53.852248    29.343517       0.733229     8.802402   
min       0.000000     0.000000    12.000000  202307.000000     1.000000   
25%     234.750000     0.000000    59.700000  202307.000000     8.000000   
50%     509.000000     0.000000    76.570000  202308.000000    15.000000   
75%     816.000000     0.000000    84.960000  202308.000000    22.000000   
max    4974.000000  2164.000000   301.470000  202309.000000    31.000000   

                 층         건축년도  
count  9272.000000  9272.000000  
mean     10.021031  2003.034944  
std       6.

In [16]:
# 결측치 비율 80% 이하인 컬럼들의 상관관계 분석
missing_pct = (train_df.isnull().sum() / len(train_df) * 100)
low_missing_cols = missing_pct[missing_pct <= 80].index.tolist()

print(f"결측치 80% 이하 컬럼 개수: {len(low_missing_cols)}")
print(f"컬럼 목록:\n{low_missing_cols}\n")

# 수치형 컬럼만 추출하여 상관관계 계산
numeric_low_missing = train_df[low_missing_cols].select_dtypes(include=['int64', 'float64'])
print(f"그 중 수치형 컬럼 개수: {len(numeric_low_missing.columns)}")
print(f"수치형 컬럼: {numeric_low_missing.columns.tolist()}\n")

# 상관관계 행렬 계산
corr_matrix = numeric_low_missing.corr()

# 삼각형 마스크 생성 (상단 절반만 표시)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# 히트맵 그리기
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(
    corr_matrix, 
    mask=mask,
    annot=True, 
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    vmin=-1, 
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8, "label": "상관계수"},
    ax=ax
)
ax.set_title('결측치 80% 이하 수치형 컬럼 상관관계 히트맵', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

fig.tight_layout()
save_plot(fig, "low_missing_80_correlation_heatmap", prefix="eda_")
plt.show()

# 높은 상관관계 (절댓값 0.5 이상, 1.0 제외) 추출
print("\n=== 높은 상관관계 (|r| >= 0.5) ===")
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) >= 0.5:
            high_corr_pairs.append({
                'Column_1': corr_matrix.columns[i],
                'Column_2': corr_matrix.columns[j],
                'Correlation': corr_val
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', key=abs, ascending=False)
    print(high_corr_df.to_string(index=False))
else:
    print("절댓값 0.5 이상의 상관관계가 없습니다.")

결측치 80% 이하 컬럼 개수: 46
컬럼 목록:
['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-수정일자', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', 'target']

그 중 수치형 컬럼 개수: 20
수치형 컬럼: ['본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target']


=== 높은 상관관계 (|r| >= 0.5) ===
              Column_1               Column_2  Correlation
              k-주거전용면적              k-관리비부과면적     0.991215
               k-전체세대수               k-주거전용면적     0.929210
               k-전체세대수   

In [15]:
# 결측치 비율 75% 이하인 컬럼들의 상관관계 분석
missing_pct = (train_df.isnull().sum() / len(train_df) * 100)
low_missing_cols_75 = missing_pct[missing_pct <= 75].index.tolist()

print(f"결측치 75% 이하 컬럼 개수: {len(low_missing_cols_75)}")
print(f"컬럼 목록:\n{low_missing_cols_75}\n")

# 수치형 컬럼만 추출하여 상관관계 계산
numeric_low_missing_75 = train_df[low_missing_cols_75].select_dtypes(include=['int64', 'float64'])
print(f"그 중 수치형 컬럼 개수: {len(numeric_low_missing_75.columns)}")
print(f"수치형 컬럼: {numeric_low_missing_75.columns.tolist()}\n")

# 상관관계 행렬 계산
corr_matrix_75 = numeric_low_missing_75.corr()

# 삼각형 마스크 생성 (상단 절반만 표시)
mask_75 = np.triu(np.ones_like(corr_matrix_75, dtype=bool))

# 히트맵 그리기
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(
    corr_matrix_75, 
    mask=mask_75,
    annot=True, 
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    vmin=-1, 
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8, "label": "상관계수"},
    ax=ax
)
ax.set_title('결측치 75% 이하 수치형 컬럼 상관관계 히트맵', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

fig.tight_layout()
save_plot(fig, "low_missing_75_correlation_heatmap", prefix="eda_")
plt.show()

# 높은 상관관계 (절댓값 0.5 이상, 1.0 제외) 추출
print("\n=== 높은 상관관계 (|r| >= 0.5) ===")
high_corr_pairs_75 = []
for i in range(len(corr_matrix_75.columns)):
    for j in range(i+1, len(corr_matrix_75.columns)):
        corr_val = corr_matrix_75.iloc[i, j]
        if abs(corr_val) >= 0.5:
            high_corr_pairs_75.append({
                'Column_1': corr_matrix_75.columns[i],
                'Column_2': corr_matrix_75.columns[j],
                'Correlation': corr_val
            })

if high_corr_pairs_75:
    high_corr_df_75 = pd.DataFrame(high_corr_pairs_75).sort_values('Correlation', key=abs, ascending=False)
    print(high_corr_df_75.to_string(index=False))
else:
    print("절댓값 0.5 이상의 상관관계가 없습니다.")

결측치 75% 이하 컬럼 개수: 15
컬럼 목록:
['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명', '등기신청일자', '거래유형', '중개사소재지', 'target']

그 중 수치형 컬럼 개수: 8
수치형 컬럼: ['본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', 'target']


=== 높은 상관관계 (|r| >= 0.5) ===
Column_1 Column_2  Correlation
 전용면적(㎡)   target     0.577041


In [26]:
# 계약년에 따른 target(가격) 변화 분석

# 계약년월에서 연도와 월 추출
train_df['계약년'] = train_df['계약년월'] // 100
train_df['계약월'] = train_df['계약년월'] % 100

print("=== 계약 기간 정보 ===")
print(f"계약년 범위: {train_df['계약년'].min()} ~ {train_df['계약년'].max()}")
print(f"계약년월 범위: {train_df['계약년월'].min()} ~ {train_df['계약년월'].max()}")
print(f"\n연도별 거래 건수:")
print(train_df['계약년'].value_counts().sort_index())

# 1. 연도별 평균 가격 추이 (통계 요약과 함께)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1-1. 연도별 평균/중앙값 가격 추이
ax = axes[0, 0]
yearly_stats = train_df.groupby('계약년')['target'].agg(['mean', 'median', 'std']).reset_index()
ax.plot(yearly_stats['계약년'], yearly_stats['mean'], marker='o', linewidth=2, markersize=8, label='평균 가격', color='#e74c3c')
ax.plot(yearly_stats['계약년'], yearly_stats['median'], marker='s', linewidth=2, markersize=8, label='중앙값 가격', color='#3498db')
ax.fill_between(yearly_stats['계약년'], 
                yearly_stats['mean'] - yearly_stats['std'], 
                yearly_stats['mean'] + yearly_stats['std'], 
                alpha=0.2, color='#e74c3c', label='±1 표준편차')
ax.set_xlabel('계약년도', fontsize=11)
ax.set_ylabel('가격 (만원)', fontsize=11)
ax.set_title('연도별 평균/중앙값 가격 추이', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xticks(yearly_stats['계약년'])

# 1-2. 연도별 거래량과 평균 가격 (이중 축)
ax1 = axes[0, 1]
ax2 = ax1.twinx()

yearly_count = train_df['계약년'].value_counts().sort_index()
color1 = '#2ecc71'
color2 = '#e74c3c'

bars = ax1.bar(yearly_count.index, yearly_count.values, alpha=0.6, color=color1, label='거래 건수')
line = ax2.plot(yearly_stats['계약년'], yearly_stats['mean'], marker='o', linewidth=2.5, 
                markersize=8, color=color2, label='평균 가격')

ax1.set_xlabel('계약년도', fontsize=11)
ax1.set_ylabel('거래 건수', fontsize=11, color=color1)
ax2.set_ylabel('평균 가격 (만원)', fontsize=11, color=color2)
ax1.set_title('연도별 거래량과 평균 가격', fontsize=13, fontweight='bold')
ax1.tick_params(axis='y', labelcolor=color1)
ax2.tick_params(axis='y', labelcolor=color2)
ax1.set_xticks(yearly_count.index)

# 범례 통합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. 연도별 가격 분포 (박스플롯)
ax = axes[1, 0]
box_data = [train_df[train_df['계약년'] == year]['target'].values for year in sorted(train_df['계약년'].unique())]
bp = ax.boxplot(box_data, labels=sorted(train_df['계약년'].unique()), 
                patch_artist=True, showmeans=True, meanline=True)
for patch in bp['boxes']:
    patch.set_facecolor('#3498db')
    patch.set_alpha(0.6)
ax.set_xlabel('계약년도', fontsize=11)
ax.set_ylabel('가격 (만원)', fontsize=11)
ax.set_title('연도별 가격 분포 (박스플롯)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 3. 연월별 평균 가격 추이 (상세)
ax = axes[1, 1]
monthly_mean = train_df.groupby('계약년월')['target'].mean().reset_index()
ax.plot(range(len(monthly_mean)), monthly_mean['target'], linewidth=1.5, color='#9b59b6')
ax.set_xlabel('계약년월 (시간 순서)', fontsize=11)
ax.set_ylabel('평균 가격 (만원)', fontsize=11)
ax.set_title('계약년월별 평균 가격 추이', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

# x축 레이블을 일부만 표시 (가독성)
step = max(1, len(monthly_mean) // 20)
xtick_positions = range(0, len(monthly_mean), step)
xtick_labels = [str(monthly_mean.iloc[i]['계약년월']) for i in xtick_positions]
ax.set_xticks(xtick_positions)
ax.set_xticklabels(xtick_labels, rotation=45, ha='right', fontsize=9)

fig.tight_layout()
save_plot(fig, "contract_year_price_analysis", prefix="eda_")
plt.show()

# 4. 통계 테이블 출력
print("\n=== 연도별 가격 통계 ===")
yearly_detailed = train_df.groupby('계약년')['target'].agg([
    ('거래건수', 'count'),
    ('평균', 'mean'),
    ('중앙값', 'median'),
    ('표준편차', 'std'),
    ('최소값', 'min'),
    ('최대값', 'max'),
    ('25%', lambda x: x.quantile(0.25)),
    ('75%', lambda x: x.quantile(0.75))
]).round(0)
print(yearly_detailed)

print("\n=== 전년 대비 가격 변화율 ===")
yearly_change = yearly_stats.copy()
yearly_change['평균_변화율(%)'] = yearly_change['mean'].pct_change() * 100
yearly_change['중앙값_변화율(%)'] = yearly_change['median'].pct_change() * 100
print(yearly_change[['계약년', '평균_변화율(%)', '중앙값_변화율(%)']].round(2))

=== 계약 기간 정보 ===
계약년 범위: 2007 ~ 2023
계약년월 범위: 200701 ~ 202306

연도별 거래 건수:
계약년
2007     58767
2008     57012
2009     73491
2010     44457
2011     54513
2012     40851
2013     67865
2014     85130
2015    119891
2016     99253
2017    104893
2018     81413
2019     74696
2020     83711
2021     43117
2022     12214
2023     17548
Name: count, dtype: int64

=== 연도별 가격 통계 ===
        거래건수        평균      중앙값     표준편차   최소값      최대값      25%       75%
계약년                                                                       
2007   58767   31970.0  25100.0  25214.0   500   490000  17100.0   37500.0
2008   57012   39560.0  33000.0  25579.0   500   570000  24800.0   46606.0
2009   73491   50163.0  40000.0  34781.0  1100   565000  28900.0   60000.0
2010   44457   49572.0  39000.0  35307.0   350   570000  28000.0   58500.0
2011   54513   45495.0  37000.0  30306.0  1000   438000  27400.0   53300.0
2012   40851   44379.0  36000.0  30565.0  2000   549913  26000.0   52500.0
2013   67865   44318.0